In [2]:
import numpy as np

In [2]:
%qtconsole

In [3]:
def generate_random_vector(N):
    return np.random.lognormal(size = N)

In [4]:
np.set_printoptions(linewidth = 150)

# Check $M^{-1}$

In [44]:
def check_M_inv(N=10, s=np.random.normal(), sigmaC=np.random.lognormal(), sigmaM=np.random.lognormal(),
                   v=None, w=None, tol=10e-13):
    if v is None:
        v = generate_random_vector(N)
    if w is None:
        w = generate_random_vector(N)
    
    V = v**2
    W = w**2
    X = v*w
    
    M = 2 * sigmaM**4 * np.identity(N) + 4 * sigmaM**2 * s**2 * np.diag(V) + 4 * sigmaM**2 * sigmaC**2 * np.diag(W) + 4 * s**2 * sigmaC**2 * np.outer(X, X)
    M_inv_num = np.linalg.inv(M)
    M_inv_anal = np.diag(1./(2 * sigmaM**4 + 4 * sigmaM**2 * s**2 * V + 4 * sigmaM**2 * sigmaC**2 * W)) - \
                    s**2 * sigmaC**2/(sigmaM**4 + 2 * s**2 * sigmaC**2 * sigmaM**2 * np.sum((V * W)/(sigmaM**2 + 2 * s**2 * V + 2 * sigmaC**2 * W))) * \
                    np.outer(X, X)/(np.outer(sigmaM**2 + 2 * s**2 * V + 2 * sigmaC**2 * W, sigmaM**2 + 2 * s**2 * V + 2 * sigmaC**2 * W))
    
    difference = np.abs(M_inv_num - M_inv_anal)
    if np.any(difference > tol):
        print('Failure')
        print('Numeric inverse:\n', M_inv_num)
        print('Analytic inverse:\n', M_inv_anal)
    else:
        print('Success')
                                    

In [46]:
check_M_inv(N=80)

Success


# Check Inverse Covariance

In [53]:
def check_inv_covar(N=10, s=np.random.normal(), sigmaC=np.random.lognormal(), sigmaM=np.random.lognormal(),
                   v=None, w=None, tol=10e-13):
    if v is None:
        v = generate_random_vector(N)
    if w is None:
        w = generate_random_vector(N)
    
    V = v**2
    W = w**2
    X = v*w
    
    covar = 2 * sigmaM**4 * np.identity(N) + 4 * sigmaM**2 * s**2 * np.diag(V) + 4 * sigmaM**2 * sigmaC**2 * np.diag(W) + 4 * s**2 * sigmaC**2 * np.outer(X, X) + 2 * sigmaC**4 * np.outer(W, W)
    inv_covar_num = np.linalg.inv(covar)
    
    # calculate semi-analytically
    M_inv = np.diag(1./(2 * sigmaM**4 + 4 * sigmaM**2 * s**2 * V + 4 * sigmaM**2 * sigmaC**2 * W)) - \
                    s**2 * sigmaC**2/(sigmaM**4 + 2 * s**2 * sigmaC**2 * sigmaM**2 * np.sum((V * W)/(sigmaM**2 + 2 * s**2 * V + 2 * sigmaC**2 * W))) * \
                    np.outer(X, X)/(np.outer(sigmaM**2 + 2 * s**2 * V + 2 * sigmaC**2 * W, sigmaM**2 + 2 * s**2 * V + 2 * sigmaC**2 * W))
    inv_covar_anal = M_inv - 2 * sigmaC**4/(1 + 2*sigmaC**4 * np.dot(W.T, np.dot(M_inv, W))) * np.dot(M_inv, np.dot(np.outer(W, W), M_inv))
    
    difference = np.abs(inv_covar_num - inv_covar_anal)
    if np.any(difference > tol):
        print('Failure')
        print('Numeric inverse:\n', inv_covar_num)
        print('Analytic inverse:\n', inv_covar_anal)
    else:
        print('Success')

In [54]:
check_inv_covar(N=60)

Success


# Check Fisher Information (Semi Analytic)

In [102]:
def check_fisher_1(N=10, s=np.random.normal(), sigmaC=np.random.lognormal(), sigmaM=np.random.lognormal(),
                   v=None, w=None, tol=10e-13):
    if v is None:
        v = generate_random_vector(N)
    if w is None:
        w = generate_random_vector(N)
    
    V = v**2
    W = w**2
    X = v*w
    
    covar = 2 * sigmaM**4 * np.identity(N) + 4 * sigmaM**2 * s**2 * np.diag(V) + 4 * sigmaM**2 * sigmaC**2 * np.diag(W) + 4 * s**2 * sigmaC**2 * np.outer(X, X) + 2 * sigmaC**4 * np.outer(W, W)
    inv_covar_num = np.linalg.inv(covar)
    fisher_num = np.dot(2 * s * V, np.dot(inv_covar_num, 2 * s * V))
    
    # calculate semi-analytically
    M_inv = np.diag(1./(2 * sigmaM**4 + 4 * sigmaM**2 * s**2 * V + 4 * sigmaM**2 * sigmaC**2 * W)) - \
                    s**2 * sigmaC**2/(sigmaM**4 + 2 * s**2 * sigmaC**2 * sigmaM**2 * np.sum((V * W)/(sigmaM**2 + 2 * s**2 * V + 2 * sigmaC**2 * W))) * \
                    np.outer(X, X)/(np.outer(sigmaM**2 + 2 * s**2 * V + 2 * sigmaC**2 * W, sigmaM**2 + 2 * s**2 * V + 2 * sigmaC**2 * W))
    fisher_anal = 4 * s**2 * (np.dot(V, np.dot(M_inv, V)) - 2 * sigmaC**4/(1 + 2 * sigmaC**4 * np.dot(W, np.dot(M_inv, W))) * (np.dot(V, np.dot(M_inv, W))**2))
    
    difference = np.abs(fisher_num - fisher_anal)
    if difference > tol:
        print('Failure')
        print('Numeric inverse:\n', fisher_num)
        print('Analytic inverse:\n', fisher_anal)
    else:
        print('Success')

In [138]:
check_fisher_1(N=10)

Success


# Check Fisher Information (Analytic)

In [124]:
def check_fisher_2(N=10, s=np.random.normal(), sigmaC=np.random.lognormal(), sigmaM=np.random.lognormal(),
                   v=None, w=None, tol=10e-13):
    if v is None:
        v = generate_random_vector(N)
    if w is None:
        w = generate_random_vector(N)
    
    V = v**2
    W = w**2
    X = v*w
    
    norm = sigmaM**2 + 2 * s**2 * V + 2 * sigmaC**2 * W
    vw40 = np.sum(v**4/norm)
    vw31 = np.sum((v**3 * w)/norm)
    vw22 = np.sum((v**2 * w**2)/norm)
    vw13 = np.sum((v * w**3)/norm)
    vw04 = np.sum(w**4/norm)
    
    covar = 2 * sigmaM**4 * np.identity(N) + 4 * sigmaM**2 * s**2 * np.diag(V) + 4 * sigmaM**2 * sigmaC**2 * np.diag(W) + 4 * s**2 * sigmaC**2 * np.outer(X, X) + 2 * sigmaC**4 * np.outer(W, W)
    inv_covar_num = np.linalg.inv(covar)
    fisher_num = np.dot(2 * s * V, np.dot(inv_covar_num, 2 * s * V))
    
    fisher_anal_denom = sigmaM**4 + sigmaM**2 * (sigmaC**4 * vw04 + 2 * s**2 * sigmaC**2 * vw22) - 2 * s**2 * sigmaC**6 * (vw13**2 - vw04 * vw22)
    fisher_anal_numer = sigmaM**4 * vw40 + sigmaC**4 * sigmaM**2 * (vw04 * vw40 - vw22**2) - 2 * s**2 * sigmaC**2 * sigmaM**2 * (vw31**2 - vw22 * vw40) - 2 * s**2 * sigmaC**6 * (vw22**3 + vw04 * vw31**2 + vw13**2 * vw40 - vw22 * (2 * vw13 * vw31 + vw04 * vw40))
    fisher_anal = (2 * s**2/sigmaM**2) * fisher_anal_numer/fisher_anal_denom
    
    print(fisher_anal)
    print(fisher_num)

In [147]:
check_fisher_2(N=10)

4.190762374914809
4.19076237491481


# 1. Check mean

In [85]:
def check_mean(N, s=np.random.normal(), sigmaC=np.random.lognormal(), sigmaM=np.random.lognormal(),
               v = None, w = None):
    if v is None:
        v = generate_random_vector(N)
    if w is None:
        w = generate_random_vector(N)
    
    # calculate mean from analytic formula
    mean_analytic = v**2 * s**2 + w**2 * sigmaI**2 + sigmaG**2
    
    # calculate mean from observations
    trials = 5000000
    xiI = np.random.normal(size = trials)
    xij = np.random.normal(size = (N, trials))
    observations = (np.reshape(v * s, (N, 1)) + np.outer(w, sigmaI * xiI) + sigmaG * xij)**2
    mean_observed = np.mean(observations, axis = 1)
    print("Analytic: ", mean_analytic)
    print("Observed: ", mean_observed)

In [86]:
check_mean(10)

Analytic:  [ 0.41145308  0.45247908  0.43866156  0.38573842  0.38023904  0.66920013  0.89111502  0.53364669  0.56671081  0.38410963]
Observed:  [ 0.411076    0.45266243  0.4389998   0.38599643  0.3803144   0.66859998  0.89145796  0.53376356  0.56662019  0.38396837]


# 2. Check covariance

In [90]:
def check_covariance(N, s=np.random.normal(), sigmaC=np.random.lognormal(), sigmaM=np.random.lognormal(),
               v = None, w = None):
    if v is None:
        v = generate_random_vector(N)
    if w is None:
        w = generate_random_vector(N)
    
    V = v**2
    W = w**2
    X = v * w
    covar_analytic = 2 * sigmaG**4 + 4 * sigmaG**2 * s**2 * np.diag(V) + 4 * sigmaG**2 * sigmaI**2 * np.diag(W) \
                + 4 * s**2 * sigmaI**2 * np.outer(X, X) + 2 * sigmaI**4 * np.outer(W, W)
    
    # calculate mean from observations
    trials = 5000000
    xiI = np.random.normal(size = trials)
    xij = np.random.normal(size = (N, trials))
    observations = (np.reshape(v * s, (N, 1)) + np.outer(w, sigmaI * xiI) + sigmaG * xij)**2
    covar_observed = np.cov(observations)
    
    print("Analytic:\n", covar_analytic)
    print("Observed:\n", covar_observed)

In [91]:
check_covariance(5)

Analytic:  [[  89.06932754   33.61228628   15.99068503   26.01611788   76.67464578]
 [  33.61228628   45.62159546   14.03037293   22.11418173   62.57849622]
 [  15.99068503   14.03037293   16.53662433   10.64450333   29.84532276]
 [  26.01611788   22.11418173   10.64450333   43.80708852   49.61394484]
 [  76.67464578   62.57849622   29.84532276   49.61394484  994.61544426]]
Observed:  [[  89.03995871   32.78497791   15.12078147   25.13182741   75.76830933]
 [  32.78497791   45.6515559    13.18718356   21.26088445   61.76302759]
 [  15.12078147   13.18718356   16.52812608    9.78471733   28.94068162]
 [  25.13182741   21.26088445    9.78471733   43.791619     48.71395035]
 [  75.76830933   61.76302759   28.94068162   48.71395035  994.48174201]]


# 3. Check Fisher information

In [110]:
def check_fisher(N, s=np.random.normal(), sigmaC=np.random.lognormal(), sigmaM=np.random.lognormal(),
               v = None, w = None):
    if v is None:
        v = generate_random_vector(N)
    if w is None:
        w = generate_random_vector(N)
    
    V = v**2
    W = w**2
    X = v * w
    covar_analytic = 2 * sigmaG**4 * np.identity(N) + 4 * sigmaG**2 * s**2 * np.diag(V) + 4 * sigmaG**2 * sigmaI**2 * np.diag(W) \
            + 4 * s**2 * sigmaI**2 * np.outer(X, X) + 2 * sigmaI**4 * np.outer(W, W)
    inv_covar = np.linalg.inv(covar_analytic)
    fisher_numeric = 4 * s**2 * np.dot(v, np.dot(inv_covar, v))
    
    norm = 1./(sigmaG**2 + 2 * s**2 * V + 2 * sigmaI**2 * W)
    vw40 = np.sum(v**4 * norm)
    vw31 = np.sum(v**3 * w * norm)
    vw22 = np.sum(v**2 * w**2 * norm)
    vw13 = np.sum(v * w**3 * norm)
    vw04 = np.sum(w**4 * norm)
    
    fisher_analytic = 4 * s**2 * (1./sigmaG**2 * vw40 - (2 * s**2 * sigmaI**2)/(sigmaG**2 + 2 * s**2 * sigmaG**2 * sigmaI**2 * vw22) * vw31**2 \
                        + (sigmaG**2 * sigmaI**4 * vw22 + 2 * s**2 * sigmaI**6 * (vw22 - 2 * vw13 * vw31))/(sigmaG**4 + sigmaG**2 * (sigmaI**4 * vw04 + 2 * s**2 * sigmaI**2 * vw22) + 2 * s**2 * sigmaI**6 *(vw04 * vw22 - 2 * vw13**2)))
    
    M = 2 * sigmaG**4 * np.identity(N) + 4 * sigmaG**2 * s**2 * np.diag(V) + 4 * sigmaG**2 * sigmaI**2 * np.diag(W) + 4 * s**2 * sigmaI**2 * np.outer(X, X)
    M_inv = np.linalg.inv(M)
    term1_analytic = 1./(2*sigmaG**2) * vw40 - (2 * s**2 * sigmaI**2)/(sigmaG**4 + 2*s**2 * sigmaI**2 * sigmaG**2 * vw22) * vw31**2
    term1_numeric = np.dot(V, np.dot(M, V))
    print(term1_analytic)
    print(term1_numeric)
    #print(fisher_analytic)
    #print(fisher_numeric)

In [111]:
check_fisher(5)

25.486827223
29923.5629445
